# Thinking 1 ： ALS都有哪些应用场景？

答：  
ALS 是显式、隐式评分稀疏矩阵分解损失函数的优化求解方式。应用场景自然包括存在用户显式，隐式行为反馈的个性化推荐场景中。   
如：用户电影评分推荐；用户书籍评分推荐；用户网购商品推荐等。

# Thinking 2 ：ALS进行矩阵分解的时候，为什么可以并行化处理

答：  
矩阵分解的目标函数：
$$Loss = \sum_{r_{ui}}{(r_{ui} -x_u^Ty_i)^2 +\lambda (\sum_N{||x_u||^2_2}+\sum_M{||y_i||^2_2})}$$
ALS交替最小二乘法：通过交替固定X、Y，最小化损失函数，以补全评分矩阵。  
- 假设优化一个显示评分稀疏矩阵：  
- 首先对 x 进行优化：  
损失函数：
$$Loss = \sum_{r_{ui}}{(r_{ui} -x_u^Ty_i)^2 + (\sum_N{||x_u||^2_2})}$$
当 $y_i$ 固定时，将其写为矩阵形式，可以对单个 $x_u$ 进行优化求解：
$$ \frac{\partial L(x_u)}{x_u} = -2Y_u(R_u-Y_ux_u)+2\lambda x_u = 0$$

$$x_u = (Y_uY_u^T+\lambda I)^{-1}Y_uR_u$$
所以，可以将 $x_u$ 进行并行化操作，在每个分区执行不同的 $x_u$ ,在操作完成时，再合并到一起。


# Thinking 3 : 梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD）

答：  
三种梯度下降方法的不同在于使用样本数量的不同：
梯度下降：
$$\hat{y_i} = b_0 +b_1 x_{i}$$
$$Loss = \sum_{i=1}^n{\hat{y_i}-y_i}^2$$
$$\partial Loss(b_1) = \sum_{i=1}^{n}{2x_{i}(b_0+b_1x_i-y_i)}$$
$$update(b_1) = b_1 - \partial Loss(b_1)$$
上述过程中，  
BGD使用全部样本进行Loss 导数的计算（假设迭代10次，则统共计算 $10*n$ 次），  
而随机梯度下降SGD每次仅使用一个样本进行运算，但计算全样本个数次 ($n$次) ，  
但 MBGD 介于两只之间（每次使用m个样本，假设计算10次，统共计算$10*m$次）

# Thinking 4 : 你阅读过和推荐系统/计算广告/预测相关的论文么？有哪些论文是你比较推荐的，可以分享到微信群中

看的推文比较多，前几天刚看了一篇公众号推文：  
清华大学王晨阳：轻量级Top-K推荐框架及相关论文介绍:  
https://mp.weixin.qq.com/s/zHiRSpIfTqEaD4PNjiWA2w  
介绍了最近发表的推荐系统论文的一些问题，以及自建的模型优点。

# Action 1 : 对MovieLens数据集进行评分预测  
>工具：可以使用Surprise或者其他  
说明使用的模型，及简要原理  

## 方法 1 ： 使用 ALS 算法，采用分布式 spark 计算用户（userid == 100）的电影评分
+ ### 模型目标函数：  
$$Loss = \sum_{r_{ui}}{(r_{ui} -x_u^Ty_i)^2 +\lambda (\sum_N{||x_u||^2_2}+\sum_M{||y_i||^2_2})}$$
+ ### Loss 最小化的求解方法：  ALS（通过先固定一个求解另一个的优化方式 ）
 
>1. 初始化Items矩阵；
2. 固定Items矩阵，求Users矩阵使得Loss函数最小化；  
>当 $y_i$ 固定时，将其写为矩阵形式，可以对单个 $x_u$ 进行优化求解：
$$ \frac{\partial L(x_u)}{x_u} = -2Y_u(R_u-Y_ux_u)+2\lambda x_u = 0$$
$$x_u = (Y_uY_u^T+\lambda I)^{-1}Y_uR_u$$
所以，可以将 $x_u$ 进行并行化操作，在每个分区执行不同的 $x_u$ ,在操作完成时，再合并到一起。  
3. 固定Users矩阵，求Items矩阵使得Loss函数最小化；
4. 重复上述2，3步过程  

+ ### spark中 ALS 参数设置：  
+ 显式行为：
>maxTter: 最小迭代次数；    
regParam：惩罚项系数$\lambda$（建议为0.01）  
rank：隐分类个数  
userCol= , 用户变量名称  
itemCol= , 商品变量名称  
ratingCol= ,评分变量名称  
- 隐式行为：
>ImplicitPrefs = true此处表明rating矩阵是隐式评分

In [1]:
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pandas as pd
import os
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-14.0.2'
os.environ['SPARK_HOME'] = 'C:\hadoop-pyspark\spark-3.0.0-bin-hadoop2.7'
sc = SparkContext()
sql_sc = SQLContext(sc)

In [10]:
pd_df_ratings = pd.read_csv('./ratings.csv')
pyspark_df_ratings = sql_sc.createDataFrame(pd_df_ratings)
pyspark_df_ratings = pyspark_df_ratings.drop('Timestamp')
print(pyspark_df_ratings.show(5, truncate=False))
# 创建ALS模型

als = ALS(rank = 8, maxIter = 15, regParam = 0.1, userCol= 'userId', itemCol='movieId', ratingCol='rating')
model = als.fit(pyspark_df_ratings)
# 对userId=100进行Top-N推荐
recommendations = model.recommendForAllUsers(5)
print(recommendations.where(recommendations.userId == 100).collect())

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|1     |2      |3.5   |
|1     |29     |3.5   |
|1     |32     |3.5   |
|1     |47     |3.5   |
|1     |50     |3.5   |
+------+-------+------+
only showing top 5 rows

None
[Row(userId=100, recommendations=[Row(movieId=40697, rating=5.0666375160217285), Row(movieId=4261, rating=5.053260803222656), Row(movieId=82931, rating=4.9245758056640625), Row(movieId=100106, rating=4.8351054191589355), Row(movieId=727, rating=4.789481163024902)])]


## spark ALS 算法参考：  
https://blog.csdn.net/pearl8899/article/details/106388812  
## Spark求解ALS的原理：  
https://blog.csdn.net/u012834750/article/details/81560971?utm_medium=distribute.pc_aggpage_search_result.none-task-blog-2~all~first_rank_v2~rank_v25-1-81560971.nonecase&utm_term=als%E7%AE%97%E6%B3%95%E5%BA%94%E7%94%A8%E5%9C%BA%E6%99%AF

## 方法 2 ：基于 surprise 的 Baseline 模型的 ALS 优化方法
+ ### BaseLine的基本思想：
Baseline算法的思想就是设立基线 $\mu$，并引入用户的偏差 $b_u$ 以及item的偏差 $b_i$
$$\hat{r_{ui}} = b_{ui} = \mu +b_u +b_i$$
+ ### BaseLine的目标函数：
$$Loss = \sum_{\hat{r}_{ui}}{(\hat{r}_{ui} -b_u^Tb_i)^2 +\lambda (\sum_N{||b_u||^2_2}+\sum_M{||b_i||^2_2})}$$
+ ### 优化方法：
ALS 交替最小二乘
+ ### 验证方法：
K 折交叉验证

In [11]:
#from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold
#import pandas as pd

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./ratings.csv', reader=reader)
train_set = data.build_full_trainset()

# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
#bsl_options = {'method': 'sgd','n_epochs': 5}
algo = BaselineOnly(bsl_options=bsl_options)
#algo = BaselineOnly()
#algo = NormalPredictor()

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Estimating biases using als...
RMSE: 0.8642
Estimating biases using als...
RMSE: 0.8645
Estimating biases using als...
RMSE: 0.8632
user: 196        item: 302        r_ui = 4.00   est = 4.10   {'was_impossible': False}


# Action 2 : Paper Reading：
>Slope one predictors for online rating-based collaborative filtering. Daniel Lemire and Anna Maclachlan, 2007. http://arxiv.org/abs/cs/0702144      
积累，总结笔记，自己的思考及idea   

# Action 3 ：设计你自己的句子生成器
>grammar = '''  
战斗 => 施法  ， 结果 。  
施法 => 主语 动作 技能   
结果 => 主语 获得 效果  
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉  
动作 => 施放 | 使用 | 召唤   
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙  
获得 => 损失 | 获得   
效果 => 数值 状态  
数值 => 1 | 1000 |5000 | 100 \n状态 => 法力 | 生命   
'''

In [13]:
import random

# 定语从句语法
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''
host = """
host = 寒暄 报数 询问 具体业务 结尾 
报数 = 我是工号 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？
"""


# 得到语法字典
def getGrammarDict(gram, linesplit = "\n", gramsplit = "="):
    #定义字典
    result = {}

    for line in gram.split(linesplit):
        # 去掉首尾空格后，如果为空则退出
        if not line.strip(): 
            continue
        expr, statement = line.split(gramsplit)
        result[expr.strip()] = [i.split() for i in statement.split("|")]
    #print(result)
    return result

# 生成句子
def generate(gramdict, target, isEng = False):
    if target not in gramdict: 
        return target
    find = random.choice(gramdict[target])
    #print(find)
    blank = ''
    # 如果是英文中间间隔为空格
    if isEng: 
        blank = ' '
    return blank.join(generate(gramdict, t, isEng) for t in find)

gramdict = getGrammarDict(host)
print(generate(gramdict,"host"))
print(generate(gramdict,"host", True))

您好我是工号781号,您需要打猎吗？
你好 我是工号 2 号 , 请问你要 喝酒 吗？


# 本章任务

In [7]:
import xlrd
data = xlrd.open_workbook('L4自测文档.xls')
#通过索引顺序获取
table = data.sheet_by_index(0)

""" 工作表中行/列的操作 """
#获取该sheet中的有效行数
nrows = table.nrows  
print(nrows)
row_index, col_index = 0, 0
# 获取某行信息
for row_index in range(2, nrows-8):
    print(table.row(row_index)[:2])
for row_index in range(nrows-7, nrows):
    print(table.row(row_index)[:2], table.row(row_index)[-2])

23
[text:'原理', text:'常用的推荐算法有哪些']
[text:'原理', text:'基于模型的协同过滤有哪些']
[text:'原理', text:'基于邻域的推荐算法']
[text:'原理', text:'什么是矩阵分解']
[text:'原理', text:'ALS进行矩阵分解的原理']
[text:'原理', text:'BGD，SGD，MBGD区别']
[text:'工具', text:'Surprise中的Baseline算法']
[text:'工具', text:'Surprise中的SlopeOne算法']
[text:'原理', text:'K折交叉验证']
[text:'原理', text:'评价指标：RMSE，MSE，FCP']
[text:'工具', text:'DataFrame与Series使用']
[text:'工具', text:'Index索引 Groupby方法']
[text:'工具', text:'map(), lambda函数']
[text:'Thinking1', text:'ALS都有哪些应用场景'] text:'能简单说明ALS的应用场景（10points）\n'
[text:'Thinking2', text:'ALS进行矩阵分解的时候，为什么可以并行化处理'] text:'理解ALS的原理，并能用自己的语言解释ALS能并行化的原因（10points)\n'
[text:'Thinking3', text:'梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD）'] text:'能说明三种梯度下降的区别（10points）'
[text:'Thinking4', text:'你阅读过和推荐系统/计算广告/预测相关的论文么？有哪些论文是你比较推荐的，可以分享到微信群中'] text:'论文推荐分享（10points）'
[text:'Action1', text:'对MovieLens数据集进行评分预测\n工具：可以使用Surprise或者其他\n说明使用的模型，及简要原理'] text:'1、完成代码，流程正确（10points）\n2、能使用Surprise或其他工具，并对使用模型的原理进行说明（10points）'
[text:'Action2'